This example shows you the basic usage of the windpowerlib. There are mainly three steps.
First you need to define your wind turbine, then get your weather data and in the last step call the windpowerlib ModelChain to generate your feedin timeseries.
Be careful with the units!

### Import necessary packages and modules

In [ ]:
__copyright__ = "Copyright oemof developer group"
__license__ = "GPLv3"

import os
import pandas as pd

from windpowerlib import modelchain
from windpowerlib import wind_turbine as wt

You can use the logging package to get logging messages from the windpowerlib. Change the logging level if you want more or less messages.

In [ ]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

### Import weather data

The function below imports the example weather data from the weather.csv file provided along with the windpowerlib. The data include wind speed at two different heights in m/s, air temperature in two different heights in K, surface roughness length in m and air pressure in Pa.

To find out which weather data in which units need to be provided in order to use the ModelChain or other functions of the windpowerlib see the individual function documentation.

In [ ]:
def read_weather_data(filename, datetime_column='Unnamed: 0',
                      **kwargs):
    r"""
    Fetches weather data from a file.

    The files are located in the example folder of the windpowerlib.

    Parameters
    ----------
    filename : string
        Filename of the weather data file.
    datetime_column : string
        Name of the datetime column of the weather DataFrame.

    Other Parameters
    ----------------
    datapath : string, optional
        Path where the weather data file is stored.
        Default: 'windpowerlib/example'.

    Returns
    -------
    pandas.DataFrame
        Contains weather data time series.

    """
    if 'datapath' not in kwargs:
        kwargs['datapath'] = os.path.join(os.path.split(
            os.path.dirname(__file__))[0], 'example')

    file = os.path.join(kwargs['datapath'], filename)
    df = pd.read_csv(file)
    return df.set_index(pd.to_datetime(df[datetime_column])).tz_localize(
        'UTC').tz_convert('Europe/Berlin').drop(datetime_column, 1)


# Read weather data from csv
weather = read_weather_data('weather.csv')

Along with the weather data you have to provide a dataframe or dictionary specifying the height for which the data applies.

In [ ]:
data_height = {
    'pressure': 0,
    'temp_air': 2,
    'v_wind': 10,
    'temp_air_2': 10,
    'v_wind_2': 80}

### Initialise wind turbine

There are two ways to initialize a WindTurbine object. You can either specify your own turbine, as done below or use

In [ ]:
wt.get_turbine_types()
turbines[turbines["turbine_id"].str.contains("VESTAS")]

In [ ]:
# Specifications of the wind turbines
enerconE126 = {
    'hub_height': 135,
    'd_rotor': 127,
    'fetch_curve': 'P',  # 'P' vor p-curve and 'cp' for cp-curve
    'turbine_name': 'ENERCON E 126 7500'}  # Turbine name as in register. Use
                                           # wind_turbine.get_turbine_types()
P_data = {0:
         3:
         5:
         10:
         15:
         20:
         25:}
# for a full list.
vestasV90 = {
    'hub_height': 105,
    'd_rotor': 90,
    'fetch_curve': 'P',
    'turbine_name': 'VESTAS V 90 3000'}

# Initialize WindTurbine objects
e126 = wt.WindTurbine(**enerconE126)
v90 = wt.WindTurbine(**vestasV90)

### Use the ModelChain to generate feedin timeseries

In [ ]:
# Specifications of the modelchain data
modelchain_data = {
    'obstacle_height': 0,
    'wind_model': 'logarithmic',       # 'logarithmic' or 'hellman'
    'rho_model': 'ideal_gas',          # 'barometric' or 'ideal_gas'
    'power_output_model': 'p_values',  # 'p_values' or 'cp_values'
    'density_corr': False,             # True or False
    'hellman_exp': None}               # Float or None

# Calculate turbine power output
mc_e126 = modelchain.ModelChain(e126, **modelchain_data).run_model(
    weather, coastDat2)
e126.power_output = mc_e126.power_output
mc_v90 = modelchain.ModelChain(v90, **modelchain_data).run_model(
    weather, coastDat2)
v90.power_output = mc_v90.power_output




### Plot results

In [ ]:
try:
    from matplotlib import pyplot as plt
except ImportError:
    plt = None
# Plot turbine power output
if plt:
    e126.power_output.plot(legend=True, label='Enercon E126')
    v90.power_output.plot(legend=True, label='Vestas V90')
    plt.show()
else:
    print(e126.power_output)
    print(v90.power_output)

# Plot power (coefficient) curves
if plt:
    if e126.cp_values is not None:
        e126.cp_values.plot(style='*', title='Enercon E126')
        plt.show()
    if e126.p_values is not None:
        e126.p_values.plot(style='*', title='Enercon E126')
        plt.show()
    if v90.cp_values is not None:
        v90.cp_values.plot(style='*', title='Vestas V90')
        plt.show()
    if v90.p_values is not None:
        v90.p_values.plot(style='*', title='Vestas V90')
        plt.show()
else:
    if e126.cp_values is not None:
        print("The cp value at a wind speed of 5 m/s: {0}".format(
            e126.cp_values.cp[5.0]))
    if e126.p_values is not None:
        print("The P value at a wind speed of 5 m/s: {0}".format(
            e126.p_values.P[5.0]))
logging.info('Done!')